## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/Churn_Modelling.csv"
file_type = "csv"

# CSV options
#df=spark.read.csv(file_location,header=True,InferSchema=True)

In [0]:
df=spark.read.csv(file_location,header=True,inferSchema=True)

In [0]:
df.printSchema()

root
-- RowNumber: integer (nullable = true)
-- CustomerId: integer (nullable = true)
-- Surname: string (nullable = true)
-- CreditScore: integer (nullable = true)
-- Geography: string (nullable = true)
-- Gender: string (nullable = true)
-- Age: integer (nullable = true)
-- Tenure: integer (nullable = true)
-- Balance: double (nullable = true)
-- NumOfProducts: integer (nullable = true)
-- HasCrCard: integer (nullable = true)
-- IsActiveMember: integer (nullable = true)
-- EstimatedSalary: double (nullable = true)
-- Exited: integer (nullable = true)

In [0]:
# File location and type
file_location = "/FileStore/tables/tips.csv"
file_type = "csv"

# CSV options
tips=spark.read.csv(file_location,header=True,inferSchema=True)

In [0]:
tips.printSchema()

root
-- total_bill: double (nullable = true)
-- tip: double (nullable = true)
-- sex: string (nullable = true)
-- smoker: string (nullable = true)
-- day: string (nullable = true)
-- time: string (nullable = true)
-- size: integer (nullable = true)

In [0]:
#sex , smoker and day are categorical features
from pyspark.ml.feature import StringIndexer

In [0]:
stin=StringIndexer(inputCols=['sex','smoker','day','time'],outputCols=['sex_indexed','smoker_indexed','day_indexed','time_indexed'])

df_new=stin.fit(tips).transform(tips)

In [0]:
df_new.columns

Out[19]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_indexed',
 'smoker_indexed',
 'day_indexed',
 'time_indexed']

In [0]:
df_new.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 23.68|3.31| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 24.59|3.61|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|
 25.29|4.71| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 8.77| 2.0| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 26.88|3.12| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 15.04|1.96| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 14.78|3.23| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 10.27|1.71| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 35.26| 5.0|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|
 15.42|1.57| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 18.43| 3.0| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|
 14.83|3.02|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|
 21.58|3.92| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|
 10.33|1.67|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|
 16.29|3.71| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|
 16.97| 3.5|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|
 20.65|3.35| Male| No|Sat|Dinner| 3| 0.0| 0.0| 0.0| 0.0|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+
only showing top 20 rows

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
vect=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed','time_indexed'],outputCol="Independent Features")

In [0]:
final_data=vect.transform(df_new)

In [0]:
final_data.show()

+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
total_bill| tip| sex|smoker|day| time|size|sex_indexed|smoker_indexed|day_indexed|time_indexed|Independent Features|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
 16.99|1.01|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|[1.01,2.0,1.0,0.0...|
 10.34|1.66| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[1.66,3.0,0.0,0.0...|
 21.01| 3.5| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[3.5,3.0,0.0,0.0,...|
 23.68|3.31| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.31,2.0,0.0,0.0...|
 24.59|3.61|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|[3.61,4.0,1.0,0.0...|
 25.29|4.71| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[4.71,4.0,0.0,0.0...|
 8.77| 2.0| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[2.0,2.0,0.0,0.0,...|
 26.88|3.12| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[3.12,4.0,0.0,0.0...|
 15.04|1.96| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.96,2.0,0.0,0.0...|
 14.78|3.23| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.23,2.0,0.0,0.0...|
 10.27|1.71| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.71,2.0,0.0,0.0...|
 35.26| 5.0|Female| No|Sun|Dinner| 4| 1.0| 0.0| 1.0| 0.0|[5.0,4.0,1.0,0.0,...|
 15.42|1.57| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[1.57,2.0,0.0,0.0...|
 18.43| 3.0| Male| No|Sun|Dinner| 4| 0.0| 0.0| 1.0| 0.0|[3.0,4.0,0.0,0.0,...|
 14.83|3.02|Female| No|Sun|Dinner| 2| 1.0| 0.0| 1.0| 0.0|[3.02,2.0,1.0,0.0...|
 21.58|3.92| Male| No|Sun|Dinner| 2| 0.0| 0.0| 1.0| 0.0|[3.92,2.0,0.0,0.0...|
 10.33|1.67|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|[1.67,3.0,1.0,0.0...|
 16.29|3.71| Male| No|Sun|Dinner| 3| 0.0| 0.0| 1.0| 0.0|[3.71,3.0,0.0,0.0...|
 16.97| 3.5|Female| No|Sun|Dinner| 3| 1.0| 0.0| 1.0| 0.0|[3.5,3.0,1.0,0.0,...|
 20.65|3.35| Male| No|Sat|Dinner| 3| 0.0| 0.0| 0.0| 0.0|(6,[0,1],[3.35,3.0])|
+----------+----+------+------+---+------+----+-----------+--------------+-----------+------------+--------------------+
only showing top 20 rows

In [0]:
final=final_data.select(['total_bill','Independent features'])

In [0]:
final.columns

Out[37]: ['total_bill', 'Independent features']

In [0]:
#linear regression
from pyspark.ml.regression import LinearRegression

train_data,test_data=final.randomSplit([0.8,0.2])
regre=LinearRegression(featuresCol="Independent features",labelCol="total_bill")
regre=regre.fit(train_data)

In [0]:
regre.coefficients

Out[38]: DenseVector([2.9912, 3.8059, -1.13, 2.179, -0.3396, -0.252])

In [0]:
regre.intercept


Out[39]: 1.395178756219162

In [0]:
pred=regre.evaluate(test_data)


In [0]:
pred.predictions.show()

+----------+--------------------+------------------+
total_bill|Independent features| prediction|
+----------+--------------------+------------------+
 5.75|[1.0,2.0,1.0,1.0,...|12.028332269852681|
 7.51|[2.0,2.0,0.0,0.0,...|14.058129150355523|
 8.52|[1.48,2.0,0.0,0.0...|12.502694090837478|
 8.77|[2.0,2.0,0.0,0.0,...|14.649748322036993|
 10.07|(6,[0,1],[1.25,2.0])| 12.74595352106075|
 10.27|[1.71,2.0,0.0,0.0...|13.782294154228852|
 10.33|[2.0,2.0,1.0,0.0,...|12.928137166182813|
 10.51|(6,[0,1],[1.25,2.0])| 12.74595352106075|
 11.17|[1.5,2.0,1.0,0.0,...|11.432526532030845|
 11.24|[1.76,2.0,0.0,1.0...|16.450515868691504|
 11.61|(6,[0,1],[3.39,2.0])| 19.14716703523117|
 11.69|[2.31,2.0,0.0,0.0...|14.985407743529743|
 12.16|[2.2,2.0,0.0,1.0,...|16.495791754560347|
 12.26|[2.0,2.0,1.0,0.0,...|12.928137166182813|
 12.43|[1.8,2.0,1.0,0.0,...|12.329892912522027|
 12.46|[1.5,2.0,0.0,0.0,...| 12.47489538738161|
 12.54|[2.5,2.0,0.0,0.0,...| 16.14535895618896|
 13.0|[2.0,2.0,1.0,1.0,...| 15.10717666697856|
 13.27|[2.5,2.0,1.0,1.0,...|17.534027623063707|
 13.42|[3.48,2.0,1.0,1.0...|19.194562993816675|
+----------+--------------------+------------------+
only showing top 20 rows

In [0]:
###performance metrics
pred.r2

Out[44]: 0.3792686405201958

In [0]:
pred.meanAbsoluteError,pred.meanSquaredError

Out[45]: (4.130690507236108, 32.283306121956954)

In [0]:
regre.save("/FileStore/tables/Linear_regression.pickle")